In [1]:
import pandas as pd
import re
import numpy as np
import os

## Model extraction procedure

In [2]:
# my parts

# os.chdir("..")
# os.chdir("experiments\\model_inference\\model_inference\\samples")
# print(os.getcwd())

In [3]:
# my parts

# datasets = ['ggr', 'vlat', 'calvi-standard', 'calvi-trick', 'holf', 'holf2']
# selected_imgs = {}

# for dataset_name in datasets:
#     current_img = dataset_name + ".txt"
#     df = pd.read_csv(current_img, header=None, names=["image_file"])
#     selected_imgs[dataset_name] = df

In [4]:
# my parts
# os.chdir("..\\..\\..\\..\\..\\downloaded_data\\questions")
# all_selected_questions = []
# for dataset_name in datasets:
#     current_question_sets = dataset_name + "_questions.csv"
#     qdf = pd.read_csv(current_question_sets)
    
#     img_list = selected_imgs[dataset_name]["image_file"]
#     filtered_qdf = qdf[qdf["image_file"].isin(img_list)]
#     all_selected_questions.append(filtered_qdf)
    
# final_questions_df = pd.concat(all_selected_questions, ignore_index=True)

In [5]:
os.getcwd()

'D:\\osu\\osu\\OSU\\OSU\\2025summer\\projects\\osfstorage-archive_OSF_ver\\analysis'

In [6]:
questions = pd.read_csv("all_questions.csv")

In [7]:
# comment since this file stores the data before compose the GPT-4V results
# new_df = pd.read_csv("all_raw_outputs.csv")
new_df = pd.read_csv("merged_final_raw_response.csv")

In [8]:
def extract_numerical_responses(word):
    """Extract all numbers from the response."""
    # remove all commas for example 1,000 -> 1000
    num_answer = str(word).replace(",", "")

    # find all negative numbers and decimals 
    pattern = r'-?\d+(\.\d+)?'
    all_numbers = list(re.finditer(pattern, num_answer))
    all_numbers = [float(num.group()) for num in all_numbers if num != '']

    return all_numbers

def find_non_floats(r):
    response_word_count = len(str(r['agent_response']).split(" "))
    
    # if only 2 or fewer words appear in the string
    if response_word_count <= 2:
        res = str(r['agent_response'])
        res = res.replace(" ", "")
        res = res.replace(",", "")
        extracted_response = extract_numerical_responses(res)
        if extracted_response:
            return extracted_response[0]
        else:
            return np.nan
    else:
        return np.nan

def find_non_mc(r):
    item = questions[
        (questions["question"] == r["question"]) &
        (questions["image_file"] == r["image_file"])                
    ].iloc[0]
    response = str(r["agent_response"]).lower()
    mc1, mc2, mc3, mc4 = item["mc1"], item["mc2"], item["mc3"], item["mc4"]

    for mc in [mc4,mc3,mc2,mc1]:
        if mc != np.nan and response.find(str(mc).lower().strip()) != -1:
            return mc

    return np.nan

def extract_valid_response(r):
    test_type = r['test_type']
    if test_type in ['holf', 'holf2', 'chartqa-test-continuous', 'chartqa-test-continuous-human']:
        return find_non_floats(r)
    elif test_type in ["vlat", 'calvi-trick', 'calvi-standard']:
        return find_non_mc(r)
    elif test_type in ['ggr']:
        item = questions[
            (questions["question"] == r["question"]) &
            (questions["image_file"] == r["image_file"])                
        ].iloc[0]

        if item['text_input_type'] == 'open_ended_question':
            return find_non_floats(r)
        elif item['text_input_type'] == 'multiple_choice_question':
            return find_non_mc(r)
        else:
            return np.nan

## Evaluate model responses 

In [9]:
def check_mc_answer(r):
    process_answer = lambda a : str(a).lower().strip()
    # try:
    #     if (r['test_type'] in ['ggr', 'vlat']) and r['agent_type'] == 'Human':
    #         return int(r['is_correct'])
    # except:
    #     print(r['test_type'], r['agent_type'])
    #     raise Error
    #     return np.nan

    # for ans
    # we do this because ggr already has floating point numerical values
    # since we convert them to strings, they don't work well
    if ((r['test_type'] in ['ggr', 'vlat'])):
        try:
            return int(float(r['correct_answer']) == float(r['agent_response']))
        except:
            pass
        

    is_correct = process_answer(r['correct_answer']) == process_answer(r['agent_response'])

    # there may be multiple correct answers so check if this  is the case
    is_correct_2 = False
    is_correct_3 = False
    try:
        is_correct_2 = process_answer(r['correct_answer_2']) == process_answer(r['agent_response'])
        is_correct_3 = process_answer(r['correct_answer_3']) == process_answer(r['agent_response'])
    except:
        pass

    return int(is_correct or is_correct_2 or is_correct_3)

def find_correct_mc(model_responses_raw):
    """
    Find if the model response has the correct multiple choice answer
    """
    model_responses = model_responses_raw.copy()
    model_responses["is_correct"] = model_responses.apply(
        check_mc_answer, axis=1
    )

    return model_responses

In [10]:
# def get_relaxed_accuracy(agent_response, correct_answer, relax_accuracy_alpha):
#     actual = abs(agent_response)
#     correct = abs(correct_answer)
    
#     return int((actual - correct) <= (correct * relax_accuracy_alpha))

def get_relaxed_accuracy(agent_response, correct_answer, relax_accuracy_alpha):
    actual = agent_response
    correct = correct_answer
    relax = correct_answer * relax_accuracy_alpha
    upper_relax = correct_answer + relax
    lower_relax = correct_answer - relax

    if correct_answer < 0:
        return int(upper_relax <= actual <= lower_relax)
    
    return int(lower_relax <= actual <= upper_relax)

def find_relaxed_accuracy(model_responses_raw, test_name, relax_acc_alpha=0):
    """
    Find relaxed accuracy
    """
    model_responses = model_responses_raw.copy()
    model_responses["agent_response"] = model_responses["agent_response"].astype(float)
    model_responses["correct_answer"] = model_responses["correct_answer"].astype(float)
    model_responses["is_correct"] = model_responses.apply(
        lambda r: get_relaxed_accuracy(r['agent_response'], r['correct_answer'], relax_acc_alpha), axis=1
    )

    return model_responses

In [11]:
def evaluate_model_response(
    model_responses, 
    test_type, 
    relax_acc_alpha=0
):
    if test_type in ['ggr', 'vlat', 'calvi-trick', 'calvi-standard']:
        model_responses = find_correct_mc(model_responses_raw=model_responses)
    else:
        model_responses = find_relaxed_accuracy(
            model_responses_raw=model_responses,
            test_name=test_type,
            relax_acc_alpha = relax_acc_alpha,
        )
    return model_responses

## Create Raw response dataframe

In [12]:
# model_responses

In [13]:
# a = raw_response_df.copy()
# a = a.dropna(subset=['agent_response'])
# a[
#     (a['agent_type'] == 'Human') 
#     & (a['test_type'] == 'vlat')
# ]['item_id'].nunique()

## Process and evaluate extracted model responses (relax $\alpha$ = 0.05)

In [14]:
# for only vlat and only the human responses, people respond as ‘$4.9’ instead of ‘$4.90’ 
# like in the other tests, so the answer is marked as incorrect
# use this dict to map to regular answers

# human_vlat_map_dict = {
#     '$5': '$5.00',
#     '$12': '$12.00',
#     '$4.9': '$4.90',
#     '$15': '$15.00',
#     '$16.7': '$16.70',
#     '$17': '$17.00',
#     '$23.4': '$23.40',
#     '$5.1': '$5.10',
#     '$45': '$45.00',
#     '$35.4': '$35.40',
#     '$5.2': '$5.20'
# }

In [15]:
# model_configs = [
#     {'model': "Salesforce/blip2-flan-t5-xl", 'top_p': 'p06', 'temp': 't1', 'color': '#5ba3cf', 'accent_color': '#8cbedd'},
#     {'model': "Salesforce/blip2-flan-t5-xxl", 'top_p': 'p1', 'temp': 't10', 'color': '#4c78a8', 'accent_color': '#81a0c2'},
#     {'model': "llava-hf/llava-1.5-7b-hf", 'top_p': 'p04', 'temp': 't1', 'color': '#f9b574', 'accent_color': '#facb9d'},
#     {'model': "llava-hf/llava-1.5-13b-hf", 'top_p': 'p1', 'temp': 't04', 'color': '#f58518', 'accent_color': '#f79d46'},
#     {'model': "liuhaotian/llava-v1.6-34b", 'top_p': 'p1', 'temp': 't04', 'color': '#BF9000', 'accent_color': '#F1C232'},
#     {'model': "google/pix2struct-chartqa-base", 'top_p': 'p08', 'temp': 't1', 'color': '#ba94c4', 'accent_color': '#c7b8d9'},
#     {'model': "google/matcha-chartqa", 'top_p': 'p04', 'temp': 't1', 'color': '#8b6db2', 'accent_color': '#a28ac1'},
#     {'model': "GPT-4V", 'top_p': 'p1', 'temp': 't02', 'color': '#c93739', 'accent_color': '#C6765E'},
#     {'model': "Human", 'top_p': 'pna', 'temp': 'tna', 'color': '#2e693b', 'accent_color': '#73D287'},
# ]

# model_configs = [
#     {'model': "GPT-4o", 'prompt': 'paper', 'color': '#c93739', 'accent_color': '#C6765E'},
#     {'model': "GPT-4o", 'prompt': 'no_ins', 'color': '#5ba3cf', 'accent_color': '#8cbedd'},
#     {'model': "GPT-4o", 'prompt': 'with_description', 'color': '#2e693b', 'accent_color': '#73D287'},
#     {'model': "GPT-5", 'prompt': 'paper', 'color': '#4c78a8', 'accent_color': '#81a0c2'},
#     {'model': "GPT-5", 'prompt': 'no_ins', 'color': '#f9b574', 'accent_color': '#facb9d'},
#     {'model': "GPT-5", 'prompt': 'with_description', 'color': '#f58518', 'accent_color': '#f79d46'},
# ]

# all_evaluated_responses_df = create_evaluated_response_dataframe(model_configs, questions, relax_acc_alpha=0.50)
# all_evaluated_responses_df.to_csv("./dataframes/all_evaluated_responses_df.csv", index=False)

In [16]:
# my parts

def create_evaluated_response_dataframe(raw_response, questions, relax_acc_alpha=0.05):
    model_responses = []
    tests = ['ggr', 'vlat', 'calvi-standard', 'calvi-trick', 'holf', 'holf2']

    
    # questions['item_id'] = questions['question'] + questions['image_file']
    
    # item_answers = questions[['item_id', 'correct_answer']]
        
    
    for test_type in tests:
        raw_response['agent_response'] = raw_response['agent_response'].astype(str)
        raw_response['agent_response'] = raw_response.apply(extract_valid_response, axis=1)
        print(test_type)
        current_df = raw_response[raw_response['test_type'] == test_type]
        current_df = evaluate_model_response(current_df, test_type, relax_acc_alpha=relax_acc_alpha)
        model_responses.append(current_df)
    
    model_responses = pd.concat(model_responses)
    model_responses['test_type'] = model_responses['test_type'].replace({
        'calvi-trick': 'calvi',
        'calvi-standard': 'calvi'
    })

    return model_responses

In [17]:
# my parts

# new_df['agent_response'] = new_df['agent_response'].astype(str)
# new_df['agent_response'] = new_df.apply(extract_valid_response, axis=1)

# def judge_is_correct(row, relax_acc_alpha=0.05):
#     if row['test_type'] in ['ggr', 'vlat', 'calvi-trick', 'calvi-standard']:
#         return check_mc_answer(row)
#     else:
#         try:
#             return get_relaxed_accuracy(float(row['agent_response']), float(row['correct_answer']), relax_acc_alpha)
#         except:
#             return np.nan

# new_df['new_is_correct'] = new_df.apply(judge_is_correct, axis=1)

In [18]:
all_evaluated_responses_df = create_evaluated_response_dataframe(new_df, questions, relax_acc_alpha=0.05)

ggr
vlat
calvi-standard
calvi-trick
holf
holf2


In [19]:
os.getcwd()

'D:\\osu\\osu\\OSU\\OSU\\2025summer\\projects\\osfstorage-archive_OSF_ver\\analysis'

In [20]:
all_evaluated_responses_df["agent_instruction_type"] = all_evaluated_responses_df["agent_type"] + "_" + all_evaluated_responses_df["instruction_type"]

In [21]:
all_evaluated_responses_df

instructions  \
0     The first part of this study consists of 13 mu...   
1     The first part of this study consists of 13 mu...   
2     The first part of this study consists of 13 mu...   
3     The first part of this study consists of 13 mu...   
4     The first part of this study consists of 13 mu...   
...                                                 ...   
4275                                                NaN   
4276                                                NaN   
4277                                                NaN   
4278                                                NaN   
4279                                                NaN   

                                            description  \
0                                                   NaN   
1                                                   NaN   
2                                                   NaN   
3                                                   NaN   
4                                                   NaN   
...                                                 ...   
4275  Chart Description: \nGrouped bar chart with no...   
4276  Chart Description: \nGrouped bar chart with no...   
4277  Chart Description: \nGrouped bar chart with no...   
4278  Chart Description: \nGrouped bar chart with no...   
4279  Chart Description: \nGrouped bar chart with no...   

                                               question  \
0     What percentage of patients recovered after ch...   
1     What percentage of patients recovered after ch...   
2     What percentage of patients recovered after ch...   
3     What percentage of patients recovered after ch...   
4     What percentage of patients recovered after ch...   
...                                                 ...   
4275  What is the average speed of aircrafts flying ...   
4276  What is the average speed of aircrafts flying ...   
4277  What is the average speed of aircrafts flying ...   
4278  What is the average speed of aircrafts flying ...   
4279  What is the average speed of aircrafts flying ...   

                                          image_file  \
0                                      ggr_item1.png   
1                                      ggr_item1.png   
2                                      ggr_item1.png   
3                                      ggr_item1.png   
4                                      ggr_item1.png   
...                                              ...   
4275  bar_bird_4_v2heightc_v3sky_v4engine_v5none.png   
4276  bar_bird_4_v2heightc_v3sky_v4engine_v5none.png   
4277  bar_bird_4_v2heightc_v3sky_v4engine_v5none.png   
4278  bar_bird_4_v2heightc_v3sky_v4engine_v5none.png   
4279  bar_bird_4_v2heightc_v3sky_v4engine_v5none.png   

                                                item_id agent_type  \
0     What percentage of patients recovered after ch...     GPT-4V   
1     What percentage of patients recovered after ch...     GPT-4V   
2     What percentage of patients recovered after ch...     GPT-4V   
3     What percentage of patients recovered after ch...     GPT-4V   
4     What percentage of patients recovered after ch...     GPT-4V   
...                                                 ...        ...   
4275  What is the average speed of aircrafts flying ...     GPT-4o   
4276  What is the average speed of aircrafts flying ...     GPT-4o   
4277  What is the average speed of aircrafts flying ...     GPT-4o   
4278  What is the average speed of aircrafts flying ...     GPT-4o   
4279  What is the average speed of aircrafts flying ...     GPT-4o   

     agent_response correct_answer test_type  is_correct  instruction_type  \
0              22.0             35       ggr           0             paper   
1              22.0             35       ggr           0             paper   
2              22.0             35       ggr           0             paper   
3              40.0             35       ggr           0             paper   
4           

In [41]:
paper_ggr_gpt5 = all_evaluated_responses_df[
    (all_evaluated_responses_df["instruction_type"] == "paper") &
    (all_evaluated_responses_df["agent_type"] == "GPT-4V") &
    (all_evaluated_responses_df["test_type"] == "holf2")
]
paper_ggr_gpt5["is_correct"].mean()

np.float64(0.023333333333333334)

In [26]:
paper_ggr_gpt5["is_correct"].mean()

np.float64(0.8363636363636363)

In [27]:
paper_holf_gpt5 = all_evaluated_responses_df[
    (all_evaluated_responses_df["instruction_type"] == "paper") &
    (all_evaluated_responses_df["agent_type"] == "GPT-5") &
    (all_evaluated_responses_df["test_type"] == "holf")
]

In [28]:
paper_holf_gpt5["is_correct"].mean()

np.float64(0.2733333333333333)

In [29]:
paper_vlat_gpt5 = all_evaluated_responses_df[
    (all_evaluated_responses_df["instruction_type"] == "with_description") &
    (all_evaluated_responses_df["agent_type"] == "GPT-5") &
    (all_evaluated_responses_df["test_type"] == "vlat")
]
paper_vlat_gpt5["is_correct"].mean()

np.float64(0.7636363636363637)

In [30]:
paper_calvi_gpt4 = all_evaluated_responses_df[
    (all_evaluated_responses_df["instruction_type"] == "with_description") &
    (all_evaluated_responses_df["agent_type"] == "GPT-5") &
    (all_evaluated_responses_df["test_type"] == "holf2")
]
paper_calvi_gpt4["is_correct"].mean()

np.float64(0.24)

In [ ]:
num_nan = paper_calvi_gpt4["agent_response"].isna().sum()
print("NaN count:", num_nan)
total = len(paper_calvi_gpt4)
print("Total:", total)
print("Prop:", num_nan / total if total > 0 else 0)

NaN count: 0
Total: 150
Prop: 0.0


In [32]:
(all_evaluated_responses_df["test_type"] == "calvi").sum()

np.int64(1000)

In [33]:
all_evaluated_responses_df["agent_response"].isna().sum()

np.int64(499)

In [34]:
# comment since this file stores the data before compose the GPT-4V results
# all_evaluated_responses_df.to_csv("all_evaluated_responses.csv", index=False, encoding="utf-8")

In [44]:
all_evaluated_responses_df[(all_evaluated_responses_df["test_type"] == "holf2")
&(all_evaluated_responses_df["agent_type"] == "GPT-4o")
&(all_evaluated_responses_df["is_correct"] == 1)]

instructions description  \
3205  You will be presented with a series of data vi...         NaN   
3644                                                NaN         NaN   
3740                                                NaN         NaN   
3744                                                NaN         NaN   

                                               question  \
3205  What is the average speed of aircrafts flying ...   
3644  What is the average speed of aircrafts flying ...   
3740  What is the average speed of aircrafts flying ...   
3744  What is the average speed of aircrafts flying ...   

                                           image_file  \
3205   bar_bird_4_v2heightc_v3sky_v4engine_v5none.png   
3644  line_bird_5_v2heightc_v3sky_v4engine_v5time.png   
3740   bar_bird_4_v2heightc_v3sky_v4engine_v5none.png   
3744   bar_bird_4_v2heightc_v3sky_v4engine_v5none.png   

                                                item_id agent_type  \
3205  What is the average speed of aircrafts flying ...     GPT-4o   
3644  What is the average speed of aircrafts flying ...     GPT-4o   
3740  What is the average speed of aircrafts flying ...     GPT-4o   
3744  What is the average speed of aircrafts flying ...     GPT-4o   

     agent_response correct_answer test_type  is_correct instruction_type  \
3205          125.0        127.338     holf2           1            paper   
3644          125.0        127.338     holf2           1   no_instruction   
3740          125.0        127.338     holf2           1   no_instruction   
3744          125.0        127.338     holf2           1   no_instruction   

     agent_instruction_type  
3205           GPT-4o_paper  
3644  GPT-4o_no_instruction  
3740  GPT-4o_no_instruction  
3744  GPT-4o_no_instruction

In [35]:
all_evaluated_responses_df.to_csv("merged_evaluated_responses.csv", index=False, encoding="utf-8")

In [17]:
# for i in range(50, 100+1, 5):
#     alpha = i / 100
#     all_evaluated_responses_df = create_evaluated_response_dataframe(model_configs, questions, relax_acc_alpha=alpha)
#     all_evaluated_responses_df.to_csv(f"./dataframes/all_evaluated_responses_df_alpha{alpha}.csv", index=False)

In [ ]:
# alpha

In [ ]:
all_evaluated_responses_df.groupby('test_type')['is_correct'].mean()

In [ ]:
all_evaluated_responses_df.groupby('test_type')['is_correct'].mean()

### Sanity checks

## Relax accuracy dataframe